# Red Neuronal para Clasificación no Lineal

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

Cree los datos de entrada y sus etiquetas:

In [ ]:
X=
y=

Para una red neuronal con dos neuronas en la capa de entrada, tres en la capa escondidad y una en la salida, inicialize los pesos y sesgos:

In [ ]:
w1 = 2 * np.random.random(size=(2,3)) - 1
w2 = 2 * np.random.random(size=(3,1)) - 1
b1 = 2 * np.random.random(size=(1,3)) - 1
b2 = 2 * np.random.random(size=(1,1)) - 1
eta=0.1

Defina la función de activación y su derivada:

In [ ]:
def sigmoid(x):
    # Definicion de la función sigmoide

def dsigmoid(y):
    # Definicion de la derivada de la función sigmoide


## Forward pass

## Backward pass

Implemente las derivadas del error respecto a los parámetros w y b:

Implemente la correción de los parámetros:

## Entrenamiento de la red neuronal

Use el esquema de "Retropropagación" para entrenar la red.

In [ ]:
# Implementación de la retropropagación

Valide del entrenamiento. Con una gráfica del error en funcion de las épocas.

In [ ]:
# Gráfica Error vs. Épocas